## Het opschonen van de feature dataset

In dit notebook wordt de dataset die opgehaald is van de KNMI (z.d) opgeschoond, zodat deze gebruikt kan worden bij het selecteren van features die uiteindelijk in een regressie model getraind worden.

In het onderzoek naar relevante features bij het voorspellen van de energieconsumptie is bepaald welke features worden meegenomen in de opgehaalde dataset van de KNMI (z.d.). In de readme staat beschreven waar het onderzoek gelezen kan worden. Er is gebleken dat er verschillende weer features relevant zijn in het voorspellen van de energieconsumptie. Vervolgens is gekeken of de KNMI data heeft van deze weer features. 

In [1]:
import pandas as pd
import datetime

De opgehaalde data is afkomstig van het KNMI (z.d.). Van het jaar 2000 t/m 2021 zijn de dagelijkse waarde opgehaald van locatie de Bilt. Locatie de Bilt is genomen omdat deze als graadmeter in Nederland wordt gezien. 

In [2]:
features = pd.read_csv('features.csv')
features

,# STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHN,FXX,TG,TN,...,DR,RH,RHX,PG,PX,PN,NG,UG,UX,UN
0,260,20000101,237,21,25,40,10,80,61,35,...,20,10,5,10240,10280,10207,7,97,99,93
1,260,20000102,205,36,37,50,20,90,73,54,...,0,-1,-1,10267,10278,10244,8,97,99,93
2,260,20000103,212,57,61,70,40,130,84,64,...,64,45,8,10203,10239,10171,8,94,97,90
3,260,20000104,231,23,38,70,10,150,64,43,...,82,109,25,10142,10176,10095,7,94,99,83
4,260,20000105,191,37,40,60,30,120,44,14,...,0,0,0,10183,10205,10159,3,90,95,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,260,20211227,141,33,33,40,30,80,56,22,...,32,12,5,9978,10015,9939,8,97,99,94
8032,260,20211228,216,29,45,70,30,170,88,73,...,64,58,23,9920,9996,9878,8,95,98,78
8033,260,20211229,231,30,37,50,20,100,95,84,...,69,50,15,10051,10097,10010,8,95,98,86
8034,260,20211230,236,64,64,80,50,150,129,100,...,22,16,9,10135,10166,10099,8,91,98,85


In [3]:
#verwijderen van # STN ,locatie, kolom
features.drop(columns=['# STN'], inplace=True)
features.head()

,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHN,FXX,TG,TN,TX,...,DR,RH,RHX,PG,PX,PN,NG,UG,UX,UN
0,20000101,237,21,25,40,10,80,61,35,81,...,20,10,5,10240,10280,10207,7,97,99,93
1,20000102,205,36,37,50,20,90,73,54,87,...,0,-1,-1,10267,10278,10244,8,97,99,93
2,20000103,212,57,61,70,40,130,84,64,96,...,64,45,8,10203,10239,10171,8,94,97,90
3,20000104,231,23,38,70,10,150,64,43,94,...,82,109,25,10142,10176,10095,7,94,99,83
4,20000105,191,37,40,60,30,120,44,14,74,...,0,0,0,10183,10205,10159,3,90,95,82


De huidige dataset van de features wordt in een vervolgstap samengevoed met de energie en inflatie dataset. Dit wordt gedaan op datum en hiervoor moeten de kolommen hetzelfde format hebben. Daarom wordt de data in deze kolom omgezet naar de notatie jaar-maand-dag en de kolomnaam wordt veranderd naar 'Datum'.

In [4]:
features['Datum'] = features['YYYYMMDD'].apply(lambda x: pd.to_datetime(str(x), format="%Y/%m"))
features.drop(columns=['YYYYMMDD'], inplace=True)
features.head()

,DDVEC,FHVEC,FG,FHX,FHN,FXX,TG,TN,TX,SQ,...,RH,RHX,PG,PX,PN,NG,UG,UX,UN,Datum
0,237,21,25,40,10,80,61,35,81,0,...,10,5,10240,10280,10207,7,97,99,93,2000-01-01
1,205,36,37,50,20,90,73,54,87,0,...,-1,-1,10267,10278,10244,8,97,99,93,2000-01-02
2,212,57,61,70,40,130,84,64,96,0,...,45,8,10203,10239,10171,8,94,97,90,2000-01-03
3,231,23,38,70,10,150,64,43,94,0,...,109,25,10142,10176,10095,7,94,99,83,2000-01-04
4,191,37,40,60,30,120,44,14,74,65,...,0,0,10183,10205,10159,3,90,95,82,2000-01-05


De namen van de kolommen worden aangepast zodat deze leesbaar en te begrijpen zijn. Het veranderen van de kolom namen is opgesplitst in twee delen. In het eerste deel wordt alleen de kolom naam veranderd. En in het tweede deel wordt de naam veranderd en worden de waarden die beschreven zijn in 0.1 gedeeld door 10, zodat de waarden in uren beschreven zijn. 

In [5]:
features.columns

Index(['DDVEC', 'FHVEC', '   FG', '  FHX', '  FHN', '  FXX', '   TG', '   TN',
       '   TX', '   SQ', '   SP', '    Q', '   DR', '   RH', '  RHX', '   PG',
       '   PX', '   PN', '   NG', '   UG', '   UX', '   UN', 'Datum'],
      dtype='object')

In [6]:
features.rename(columns = {'DDVEC':'wind_richting', '   SP':'zon_perc', '    Q':'zon_straling', 
                           '   NG':'bewolking', '   UG':'dag_vocht', '   UX':'max_vocht', 
                           '   UN':'min_vocht' }, inplace = True)
features

,wind_richting,FHVEC,FG,FHX,FHN,FXX,TG,TN,TX,SQ,...,RH,RHX,PG,PX,PN,bewolking,dag_vocht,max_vocht,min_vocht,Datum
0,237,21,25,40,10,80,61,35,81,0,...,10,5,10240,10280,10207,7,97,99,93,2000-01-01
1,205,36,37,50,20,90,73,54,87,0,...,-1,-1,10267,10278,10244,8,97,99,93,2000-01-02
2,212,57,61,70,40,130,84,64,96,0,...,45,8,10203,10239,10171,8,94,97,90,2000-01-03
3,231,23,38,70,10,150,64,43,94,0,...,109,25,10142,10176,10095,7,94,99,83,2000-01-04
4,191,37,40,60,30,120,44,14,74,65,...,0,0,10183,10205,10159,3,90,95,82,2000-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,141,33,33,40,30,80,56,22,90,10,...,12,5,9978,10015,9939,8,97,99,94,2021-12-27
8032,216,29,45,70,30,170,88,73,107,12,...,58,23,9920,9996,9878,8,95,98,78,2021-12-28
8033,231,30,37,50,20,100,95,84,121,0,...,50,15,10051,10097,10010,8,95,98,86,2021-12-29
8034,236,64,64,80,50,150,129,100,139,0,...,16,9,10135,10166,10099,8,91,98,85,2021-12-30


In [7]:
features.columns

Index(['wind_richting', 'FHVEC', '   FG', '  FHX', '  FHN', '  FXX', '   TG',
       '   TN', '   TX', '   SQ', 'zon_perc', 'zon_straling', '   DR', '   RH',
       '  RHX', '   PG', '   PX', '   PN', 'bewolking', 'dag_vocht',
       'max_vocht', 'min_vocht', 'Datum'],
      dtype='object')

In [8]:
features['vector_wind'] = features['FHVEC'].div(10)
features['wind'] = features['   FG'].div(10)
features['max_wind'] = features['  FHX'].div(10)
features['min_wind'] = features['  FHN'].div(10)
features['max_windstoot'] = features['  FXX'].div(10)
features['gem_temp'] = features['   TG'].div(10)
features['min_temp'] = features['   TN'].div(10)
features['max_temp'] = features['   TX'].div(10)
features['zon_uren'] = features['   SQ'].div(10)
features['duur_neerslag'] = features['   DR'].div(10)
features['dag_neerslag'] = features['   RH'].div(10)
features['max_neerslag'] = features['  RHX'].div(10)
features['dag_luchtdruk'] = features['   PG'].div(10)
features['max_luchtdruk'] = features['   PX'].div(10)
features['min_luchtdruk'] = features['   PN'].div(10)
features.drop(columns=['FHVEC', '   FG', '  FHX', '  FHN', '  FXX', '   TG',
       '   TN', '   TX', '   SQ', '   DR', '   RH', '  RHX', '   PG', '   PX', '   PN'], inplace=True)
features

,wind_richting,zon_perc,zon_straling,bewolking,dag_vocht,max_vocht,min_vocht,Datum,vector_wind,wind,...,gem_temp,min_temp,max_temp,zon_uren,duur_neerslag,dag_neerslag,max_neerslag,dag_luchtdruk,max_luchtdruk,min_luchtdruk
0,237,0,93,7,97,99,93,2000-01-01,2.1,2.5,...,6.1,3.5,8.1,0.0,2.0,1.0,0.5,1024.0,1028.0,1020.7
1,205,0,68,8,97,99,93,2000-01-02,3.6,3.7,...,7.3,5.4,8.7,0.0,0.0,-0.1,-0.1,1026.7,1027.8,1024.4
2,212,0,35,8,94,97,90,2000-01-03,5.7,6.1,...,8.4,6.4,9.6,0.0,6.4,4.5,0.8,1020.3,1023.9,1017.1
3,231,0,30,7,94,99,83,2000-01-04,2.3,3.8,...,6.4,4.3,9.4,0.0,8.2,10.9,2.5,1014.2,1017.6,1009.5
4,191,82,347,3,90,95,82,2000-01-05,3.7,4.0,...,4.4,1.4,7.4,6.5,0.0,0.0,0.0,1018.3,1020.5,1015.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,141,13,162,8,97,99,94,2021-12-27,3.3,3.3,...,5.6,2.2,9.0,1.0,3.2,1.2,0.5,997.8,1001.5,993.9
8032,216,15,107,8,95,98,78,2021-12-28,2.9,4.5,...,8.8,7.3,10.7,1.2,6.4,5.8,2.3,992.0,999.6,987.8
8033,231,0,87,8,95,98,86,2021-12-29,3.0,3.7,...,9.5,8.4,12.1,0.0,6.9,5.0,1.5,1005.1,1009.7,1001.0
8034,236,0,92,8,91,98,85,2021-12-30,6.4,6.4,...,12.9,10.0,13.9,0.0,2.2,1.6,0.9,1013.5,1016.6,1009.9


Omdat de energiedata alleen beschikbaar is per maand moeten alle waarde omgezet worden naar maandelijkse data. Dit is gedaan door voor iedere feature het gemiddelde per maand te berekenen.

In [9]:
features_maand = features.groupby(by = features['Datum'].dt.to_period("M")).mean()
features_maand

,wind_richting,zon_perc,zon_straling,dag_vocht,max_vocht,min_vocht,vector_wind,wind,max_wind,min_wind,...,gem_temp,min_temp,max_temp,zon_uren,duur_neerslag,dag_neerslag,max_neerslag,dag_luchtdruk,max_luchtdruk,min_luchtdruk
Datum,,,,,,,,,,,,,,,,,,,,,
2000-01,223.612903,21.838710,201.774194,89.258065,95.806452,80.129032,3.980645,4.219355,5.967742,2.483871,...,4.329032,1.729032,6.561290,1.806452,1.403226,1.306452,0.390323,1023.045161,1026.638710,1019.390323
2000-02,237.482759,36.827586,484.310345,83.827586,94.793103,67.793103,4.296552,4.748276,6.724138,2.793103,...,5.855172,2.537931,8.941379,3.682759,2.855172,3.510345,1.058621,1017.672414,1022.613793,1012.151724
2000-03,202.483871,21.967742,629.258065,85.612903,95.774194,69.709677,3.677419,3.980645,5.903226,2.032258,...,6.796774,3.361290,10.112903,2.583871,2.038710,2.535484,0.867742,1019.225806,1022.429032,1015.967742
2000-04,166.400000,37.066667,1263.600000,76.433333,94.266667,57.533333,2.840000,3.553333,5.700000,1.600000,...,9.986667,5.186667,14.753333,5.116667,1.266667,1.286667,0.590000,1007.750000,1011.353333,1004.216667
2000-05,158.451613,43.354839,1701.903226,74.838710,94.064516,53.612903,2.900000,3.496774,5.451613,1.580645,...,14.732258,9.274194,20.022581,6.780645,1.851613,2.716129,0.909677,1014.806452,1017.109677,1011.929032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08,225.741935,36.258065,1391.612903,81.419355,96.645161,62.548387,2.677419,2.958065,4.612903,1.354839,...,16.935484,12.735484,20.906452,5.325806,1.038710,2.948387,1.632258,1015.993548,1018.061290,1013.829032
2021-09,164.500000,46.433333,1165.900000,81.300000,97.066667,60.200000,2.160000,2.510000,4.166667,1.100000,...,15.930000,10.826667,21.186667,5.940000,0.366667,0.913333,0.386667,1018.666667,1020.956667,1016.546667
2021-10,200.580645,36.774194,619.709677,85.774194,96.516129,69.225806,3.045161,3.335484,5.258065,1.741935,...,11.612903,7.541935,15.558065,3.945161,2.574194,4.654839,1.700000,1016.035484,1019.864516,1011.967742


In [10]:
#maandelijkse dataset opslaan
features_maand.to_csv('features_maand.csv')

## Bronnenlijst
KNMI. (sd). Dagwaarnemingen. Opgehaald van KNMI: https://www.daggegevens.knmi.nl/klimatologie/daggegevens